# **Install required libraries**

In [1]:
!pip install opencv-python
!pip install opencv-python-headless


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/PRODIGY/train.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./train')


# **Preprocess the data**

In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_images_with_downsampling(folder_path, img_size, num_samples_per_class):
    data = []
    labels = []
    cat_count = 0
    dog_count = 0

    for img_name in os.listdir(folder_path):
        if 'cat' in img_name and cat_count < num_samples_per_class:
            label = 0
            cat_count += 1
        elif 'dog' in img_name and dog_count < num_samples_per_class:
            label = 1
            dog_count += 1
        else:
            continue

        img_path = os.path.join(folder_path, img_name)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            img = img.flatten()

            data.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error reading {img_name}: {e}")

    return np.array(data), np.array(labels)

FOLDER_PATH = '/content/train/train'
IMG_SIZE = 64
NUM_SAMPLES_PER_CLASS = 2500
X, y = load_images_with_downsampling(FOLDER_PATH, IMG_SIZE, NUM_SAMPLES_PER_CLASS)
X = X / 255.0
print(f"Dataset Size: {X.shape[0]} images")

Dataset Size: 5000 images


# **Split data**

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Set Size: {X_train.shape[0]}")
print(f"Testing Set Size: {X_test.shape[0]}")


Training Set Size: 4000
Testing Set Size: 1000


# **Train the SVM Model**

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

print("Training the SVM...")
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train, y_train)
print("Training Complete!")

y_pred = svm.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Training the SVM...
Training Complete!

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.65      0.64       501
           1       0.64      0.63      0.64       499

    accuracy                           0.64      1000
   macro avg       0.64      0.64      0.64      1000
weighted avg       0.64      0.64      0.64      1000

Accuracy: 64.10%
